<a href="https://colab.research.google.com/github/danielbauer1979/ML_656/blob/main/Module9_LasVegasExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd 
import seaborn as sns
import graphviz
import pydot
from io import StringIO  

from sklearn.preprocessing import StandardScaler, scale
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, multilabel_confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from tensorflow.keras import layers

# Las Vegas Dataset

In [2]:
!git clone https://github.com/danielbauer1979/ML_656.git

Cloning into 'ML_656'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 107 (delta 47), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (107/107), 21.69 MiB | 3.96 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [103]:
lasvegas = pd.read_csv('ML_656/LasVegasTripAdvisorReviews.csv')
lasvegas.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3


In [104]:
numerics = list(lasvegas.select_dtypes(include=['int64']).columns)
numerics.remove('Hotel stars')
numerics.remove('Score')
factors = list(lasvegas.select_dtypes(include=['object']).columns)
factors.append('Hotel stars')
factors.remove('User country')
factors.remove('Hotel name')

In [105]:
lasvegas_numcols = lasvegas[numerics]
lasvegas_faccols = lasvegas[factors]
dummies = pd.get_dummies(lasvegas_faccols, drop_first=True)
lasvegas_numcols_sc_0 = scale(lasvegas_numcols)
lasvegas_numcols_sc = pd.DataFrame(data=lasvegas_numcols_sc_0, columns = lasvegas_numcols.columns, index = dummies.index)
lasvegas_sc = pd.concat([lasvegas_numcols_sc, dummies], axis = 1)
lasvegas_sc = pd.concat([lasvegas_sc, lasvegas['Score']], axis =1)
lasvegas_sc = lasvegas_sc.rename(columns={"Score":"Y"})
lasvegas_sc.head()

,Nr. reviews,Nr. hotel reviews,Helpful votes,Hotel stars,Period of stay_Jun-Aug,Period of stay_Mar-May,Period of stay_Sep-Nov,Traveler type_Couples,Traveler type_Families,Traveler type_Friends,Traveler type_Solo,Pool_YES,Gym_YES,Tennis court_YES,Spa_YES,Casino_YES,Free internet_YES,Y
0,-0.495595,-0.502370,-0.386857,3,0,0,0,0,0,1,0,0,1,0,0,1,1,5
1,0.945905,0.207912,0.892215,3,0,0,0,0,0,0,0,0,1,0,0,1,1,3
2,-0.161914,-0.293464,-0.139295,3,0,1,0,0,1,0,0,0,1,0,0,1,1,5
3,-0.455553,-0.377026,-0.366227,3,0,1,0,0,0,1,0,0,1,0,0,1,1,4
4,-0.575678,-0.460589,-0.613789,3,0,1,0,0,0,0,1,0,1,0,0,1,1,4


In [106]:
np.random.seed(42)
train, test = train_test_split(lasvegas_sc, test_size = 0.3)
X_train = train.drop(columns = ['Y']).values
y_train = train['Y'].values
X_test = test.drop(columns = ['Y']).values
y_test = test['Y'].values

Neural, once again

In [107]:
inputs = keras.Input(shape=(17,))
x = layers.Dense(3, activation="relu", name="dense_1")(inputs)
x = layers.Dense(2, activation="relu", name="dense_2")(x)
outputs = layers.Dense(1, name="predictions")(x)
lasvegas_reg_nnet = keras.Model(inputs=inputs, outputs=outputs)
lasvegas_reg_nnet.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.004),
  # Loss function to minimize
  loss='mean_squared_error',
  # List of metrics to monitor
)

In [108]:
history_reg = lasvegas_reg_nnet.fit(
  X_train,
  y_train,
  batch_size=50,
  epochs=850
)

Epoch 1/850
8/8 [==============================] - 1s 2ms/step - loss: 10.1699
Epoch 2/850
8/8 [==============================] - 0s 2ms/step - loss: 6.5968
Epoch 3/850
8/8 [==============================] - 0s 2ms/step - loss: 3.8395
Epoch 4/850
8/8 [==============================] - 0s 2ms/step - loss: 2.0846
Epoch 5/850
8/8 [==============================] - 0s 2ms/step - loss: 1.3215
Epoch 6/850
8/8 [==============================] - 0s 2ms/step - loss: 1.2018
Epoch 7/850
8/8 [==============================] - 0s 2ms/step - loss: 1.2202
Epoch 8/850
8/8 [==============================] - 0s 3ms/step - loss: 1.1953
Epoch 9/850
8/8 [==============================] - 0s 3ms/step - loss: 1.1668
Epoch 10/850
8/8 [==============================] - 0s 3ms/step - loss: 1.1509
Epoch 11/850
8/8 [==============================] - 0s 3ms/step - loss: 1.1472
Epoch 12/850
8/8 [==============================] - 0s 3ms/step - loss: 1.1487
Epoch 13/850
8/8 [==============================] - 0s 3ms/s

In [109]:
lasvegas_nnet_reg_preds = lasvegas_reg_nnet.predict(X_test)
lasvegas_nnet_reg_preds = lasvegas_nnet_reg_preds.reshape((152,))
np.corrcoef(lasvegas_nnet_reg_preds,y_test)[0,1] * np.corrcoef(lasvegas_nnet_reg_preds,y_test)[0,1]

0.11642934395125193

In [110]:
lasvegas_nnet_reg_preds

array([4.2986174, 4.465731 , 4.253001 , 4.272933 , 3.7802973, 4.3483033,
       4.066262 , 4.2824855, 4.090468 , 4.0305037, 4.3492064, 3.010138 ,
       4.405708 , 4.4803667, 4.2149243, 4.1748056, 4.270822 , 4.440194 ,
       4.6634336, 4.2465963, 4.5240083, 4.2909265, 4.0535803, 4.0567155,
       4.4656224, 4.4490423, 4.213475 , 4.3443933, 4.285159 , 4.261263 ,
       4.269583 , 4.3510976, 3.995545 , 4.08815  , 4.2069592, 4.2593393,
       2.9881642, 4.2846684, 4.2174063, 4.4551086, 4.1182513, 4.2644715,
       4.588353 , 3.4517746, 4.019611 , 3.97514  , 4.1484127, 3.55871  ,
       4.2510805, 3.3442512, 3.2859504, 4.36133  , 3.0821393, 4.3696337,
       4.4647007, 4.345966 , 4.446261 , 3.945521 , 4.4225225, 4.34083  ,
       3.4708169, 3.4435425, 4.2870865, 2.9754086, 4.3302994, 4.462653 ,
       4.3095584, 4.232334 , 3.879749 , 4.375394 , 4.1617002, 4.4454083,
       4.223151 , 3.9954927, 4.468731 , 4.3822255, 4.3774767, 4.6008444,
       3.7923248, 4.076218 , 3.2690141, 4.4306016, 

In [111]:
lasvegas_nnet_reg_preds_clss = np.round(lasvegas_nnet_reg_preds)
lasvegas_nnet_reg_preds_clss

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3., 4., 4., 4., 4., 4.,
       4., 5., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 3., 4., 4., 4., 4., 4., 5., 3., 4., 4., 4., 4., 4., 3., 3.,
       4., 3., 4., 4., 4., 4., 4., 4., 4., 3., 3., 4., 3., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 3., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4.,
       4., 3., 4., 4., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 3., 4., 3., 4., 4., 5., 5., 5., 3., 4., 4., 4.],
      dtype=float32)

In [112]:
confusion_matrix(y_test, lasvegas_nnet_reg_preds_clss)

array([[ 0,  0,  1,  2,  0],
       [ 0,  0,  3,  7,  1],
       [ 0,  0,  5, 12,  1],
       [ 0,  0,  4, 42,  5],
       [ 0,  0,  2, 62,  5]])

Multinomial Regression

In [113]:
multin_model = LogisticRegression(fit_intercept=True, multi_class='multinomial',max_iter=10000).fit(X_train,y_train)

In [114]:
y_mnmod_pred = multin_model.predict(X_test)
y_mnmod_pred

array([5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5,
       4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 5, 5, 5, 5, 5, 4,
       5, 4, 4, 4, 5, 4, 4, 5, 4, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 4, 5, 5,
       5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 4, 4, 5, 4, 5, 3, 5, 5, 4, 5, 5, 5, 3, 4, 5, 4, 4, 5, 4,
       4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 4, 4, 5, 5, 5, 5, 5,
       4, 5, 5, 5, 5, 4, 5, 5, 3, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 5])

In [115]:
y_test

array([5, 5, 4, 3, 5, 4, 5, 5, 4, 5, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5,
       5, 5, 4, 5, 3, 5, 4, 4, 5, 5, 4, 5, 3, 2, 5, 4, 2, 4, 3, 4, 5, 4,
       5, 5, 3, 2, 4, 5, 3, 5, 4, 4, 5, 3, 5, 5, 5, 5, 3, 2, 4, 3, 4, 5,
       5, 3, 2, 2, 4, 3, 5, 3, 4, 4, 4, 5, 4, 4, 1, 5, 4, 5, 4, 5, 4, 5,
       4, 5, 5, 5, 1, 4, 5, 5, 5, 5, 2, 4, 4, 5, 5, 4, 5, 5, 2, 5, 5, 4,
       3, 4, 4, 5, 5, 5, 4, 4, 5, 4, 2, 5, 5, 5, 3, 4, 5, 4, 5, 5, 5, 5,
       3, 5, 5, 5, 5, 4, 4, 4, 4, 1, 3, 5, 4, 4, 2, 3, 2, 5, 4, 4])

In [116]:
np.corrcoef(y_mnmod_pred,y_test)[0,1] * np.corrcoef(y_mnmod_pred,y_test)[0,1]

0.07780061193125624

In [117]:
confusion_matrix(y_test, y_mnmod_pred)

array([[ 0,  0,  0,  3,  0],
       [ 0,  0,  0,  5,  6],
       [ 0,  0,  2,  8,  8],
       [ 0,  0,  2,  9, 40],
       [ 0,  0,  2, 10, 57]])

In [118]:
multilabel_confusion_matrix(y_test, y_mnmod_pred)

array([[[149,   0],
        [  3,   0]],

       [[141,   0],
        [ 11,   0]],

       [[130,   4],
        [ 16,   2]],

       [[ 75,  26],
        [ 42,   9]],

       [[ 29,  54],
        [ 12,  57]]])

In [119]:
accuracy_score(y_test, y_mnmod_pred)

0.4473684210526316

Neural Nets

In [120]:
dummy_y_train = np_utils.to_categorical(y_train)

In [121]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352 entries, 5 to 102
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nr. reviews             352 non-null    float64
 1   Nr. hotel reviews       352 non-null    float64
 2   Helpful votes           352 non-null    float64
 3   Hotel stars             352 non-null    int64  
 4   Period of stay_Jun-Aug  352 non-null    uint8  
 5   Period of stay_Mar-May  352 non-null    uint8  
 6   Period of stay_Sep-Nov  352 non-null    uint8  
 7   Traveler type_Couples   352 non-null    uint8  
 8   Traveler type_Families  352 non-null    uint8  
 9   Traveler type_Friends   352 non-null    uint8  
 10  Traveler type_Solo      352 non-null    uint8  
 11  Pool_YES                352 non-null    uint8  
 12  Gym_YES                 352 non-null    uint8  
 13  Tennis court_YES        352 non-null    uint8  
 14  Spa_YES                 352 non-null    ui

In [122]:
inputs = keras.Input(shape=(17,))
x = layers.Dense(3, activation="relu", name="dense_1")(inputs)
x = layers.Dense(2, activation="relu", name="dense_2")(x)
outputs = layers.Dense(6, activation='softmax', name="predictions")(x)
lasvegas_nnet = keras.Model(inputs=inputs, outputs=outputs)
lasvegas_nnet.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.005),
  # Loss function to minimize
  loss='categorical_crossentropy',
  # List of metrics to monitor
  metrics=['accuracy'],
)

In [123]:
history = lasvegas_nnet.fit(
  X_train,
  dummy_y_train,
  batch_size=50,
  epochs=1000
)

Epoch 1/1000
8/8 [==============================] - 1s 3ms/step - loss: 2.0158 - accuracy: 0.4489
Epoch 2/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.7464 - accuracy: 0.4489
Epoch 3/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.6430 - accuracy: 0.4517
Epoch 4/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.6081 - accuracy: 0.4517
Epoch 5/1000
8/8 [==============================] - 0s 3ms/step - loss: 1.5635 - accuracy: 0.4545
Epoch 6/1000
8/8 [==============================] - 0s 3ms/step - loss: 1.5244 - accuracy: 0.4545
Epoch 7/1000
8/8 [==============================] - 0s 2ms/step - loss: 1.4935 - accuracy: 0.4545
Epoch 8/1000
8/8 [==============================] - 0s 3ms/step - loss: 1.4679 - accuracy: 0.4545
Epoch 9/1000
8/8 [==============================] - 0s 3ms/step - loss: 1.4452 - accuracy: 0.4545
Epoch 10/1000
8/8 [==============================] - 0s 3ms/step - loss: 1.4248 - accuracy: 0.4574
Epoch 11/1000
8/8 [

In [124]:
lasvegas_nnet_val_preds = lasvegas_nnet.predict(X_test)
lasvegas_nnet_val_preds_class = np.argmax(lasvegas_nnet_val_preds, axis = 1)
lasvegas_nnet_val_preds_class

array([4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 4, 5, 4, 4, 5, 5, 5, 5, 5, 4,
       4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 4, 5, 5,
       5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       4, 5, 5, 4, 4, 4, 4, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4,
       5, 5, 4, 5, 4, 5, 4, 4, 5, 5, 4, 4, 5, 5, 4, 4, 5, 5, 4, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 5, 5, 4, 4, 4, 5, 5, 5, 5, 5, 4, 4, 5, 4])

In [125]:
y_test

array([5, 5, 4, 3, 5, 4, 5, 5, 4, 5, 5, 3, 4, 4, 5, 4, 5, 4, 5, 5, 4, 5,
       5, 5, 4, 5, 3, 5, 4, 4, 5, 5, 4, 5, 3, 2, 5, 4, 2, 4, 3, 4, 5, 4,
       5, 5, 3, 2, 4, 5, 3, 5, 4, 4, 5, 3, 5, 5, 5, 5, 3, 2, 4, 3, 4, 5,
       5, 3, 2, 2, 4, 3, 5, 3, 4, 4, 4, 5, 4, 4, 1, 5, 4, 5, 4, 5, 4, 5,
       4, 5, 5, 5, 1, 4, 5, 5, 5, 5, 2, 4, 4, 5, 5, 4, 5, 5, 2, 5, 5, 4,
       3, 4, 4, 5, 5, 5, 4, 4, 5, 4, 2, 5, 5, 5, 3, 4, 5, 4, 5, 5, 5, 5,
       3, 5, 5, 5, 5, 4, 4, 4, 4, 1, 3, 5, 4, 4, 2, 3, 2, 5, 4, 4])

In [126]:
accuracy_score(y_test, lasvegas_nnet_val_preds_class)

0.4605263157894737

In [127]:
confusion_matrix(y_test, lasvegas_nnet_val_preds_class)

array([[ 0,  0,  0,  3,  0],
       [ 0,  0,  0,  5,  6],
       [ 0,  0,  0,  9,  9],
       [ 0,  0,  0, 19, 32],
       [ 0,  0,  0, 18, 51]])

In [128]:
np.corrcoef(lasvegas_nnet_val_preds_class,y_test)[0,1] * np.corrcoef(lasvegas_nnet_val_preds_class,y_test)[0,1]

0.054090440753236095

In [129]:
accuracy_score(y_test, lasvegas_nnet_reg_preds_clss)

0.34210526315789475